# Understanding Embedding Spaces

## Imports and definitions

In [58]:
import json
import pprint
import pandas as pd
from IPython.display import display
! pip install av
! pip install yt-dlp
import yt_dlp
import random
import av
import av.datasets
from pathlib import Path

## Donwloading the videos

In [59]:
# Load the metadata file
with open("activity_net.v1-3.min.json", "r") as f:
    dataset = json.load(f)

with open("train.json", "r") as f:
    timestamps_db = json.load(f)

In [60]:
# Extract video information
videos = dataset.get('database', {})

# Filter videos with "subset": "training"
training_videos = {video_id: video_info for video_id, video_info in videos.items() if video_info.get('subset') == 'training'}

# Sort videos by the number of moments (annotations)
sorted_videos = sorted(training_videos.items(), key=lambda x: len(x[1].get('annotations', [])), reverse=True)

# Select the top 10 videos
top_10_videos = sorted_videos[:11]

# Print the top 10 video IDs and their moment counts
for video_id, video_info in top_10_videos:
    print(f"Video ID: {video_id}, Moments: {len(video_info.get('annotations', []))}")


Video ID: o1WPnnvs00I, Moments: 23
Video ID: oGwn4NUeoy8, Moments: 23
Video ID: VEDRmPt_-Ms, Moments: 20
Video ID: qF3EbR8y8go, Moments: 19
Video ID: DLJqhYP-C0k, Moments: 18
Video ID: jBvGvVw3R-Q, Moments: 18
Video ID: -ap649M020k, Moments: 15
Video ID: lxMSX0Y0b8I, Moments: 15
Video ID: 8JKK6JayDeA, Moments: 15
Video ID: LAkaYds-8X0, Moments: 13
Video ID: 74MEdLMrXAQ, Moments: 13


In [61]:
def download_video(url, output_path='downloads/'):
    ydl_opts = {
        'outtmpl': f'{output_path}/%(id)s - %(title)s.%(ext)s',
        'format': 'mp4/bestvideo+bestaudio',
        'merge_output_format': 'mp4'
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

In [62]:
count = 0
for video_id, video_info in top_10_videos:
    video_url = video_info.get('url')
    if video_url and "youtube.com" in video_url and count < 11:
        try:
            download_video(video_url)
            print(f"Downloaded: {video_id}")
            count += 1
        except Exception as e:
            print(f"Failed to download {video_id}: {e}")
    else:
        print(f"Invalid or unsupported URL for video ID or already gor 10 videos: {video_id}")

[youtube] Extracting URL: https://www.youtube.com/watch?v=o1WPnnvs00I
[youtube] o1WPnnvs00I: Downloading webpage
[youtube] o1WPnnvs00I: Downloading tv client config
[youtube] o1WPnnvs00I: Downloading player 73381ccc-main
[youtube] o1WPnnvs00I: Downloading tv player API JSON
[youtube] o1WPnnvs00I: Downloading ios player API JSON
[youtube] o1WPnnvs00I: Downloading m3u8 information
[info] o1WPnnvs00I: Downloading 1 format(s): 18
[download] downloads\o1WPnnvs00I - Irish March (Fife & Flute) – Wouter Kellerman (Live).mp4 has already been downloaded
[download] 100% of   10.19MiB
Downloaded: o1WPnnvs00I
[youtube] Extracting URL: https://www.youtube.com/watch?v=oGwn4NUeoy8
[youtube] oGwn4NUeoy8: Downloading webpage
[youtube] oGwn4NUeoy8: Downloading tv client config
[youtube] oGwn4NUeoy8: Downloading player 73381ccc-main
[youtube] oGwn4NUeoy8: Downloading tv player API JSON
[youtube] oGwn4NUeoy8: Downloading ios player API JSON
[youtube] oGwn4NUeoy8: Downloading m3u8 information
[info] oGwn4NU

ERROR: [youtube] LAkaYds-8X0: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Failed to download LAkaYds-8X0: ERROR: [youtube] LAkaYds-8X0: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
[youtube] Extracting URL: https://www.youtube.com/watch?v=74MEdLMrXAQ
[youtube] 74MEdLMrXAQ: Downloading webpage
[youtube] 74MEdLMrXAQ: Downloading tv client config
[youtube] 74MEdLMrXAQ: Downloading player 73381ccc-main
[youtube] 74MEdLMrXAQ: Downloading tv player API JSON
[youtube] 74MEdLMrXAQ: Downloading ios player API JSON
[youtube] 74MEdLMrXAQ: Downloading m3u8 information
[info] 74MEdLMrXAQ: Downloading 1 format(s): 18
[download] downloads\74MEdLMrXAQ - Basic futsal rules.mp4 has already been downloaded
[download] 100% of   14.21MiB
Downloaded

Building a clean list of metadata entries from the downloaded videos

In [63]:
# video_db = dataset["database"]

# Get the list of renamed video files
download_folder = Path("downloads")
downloaded_files = list(download_folder.glob("*.mp4"))

# Extract the video ID from the new filenames
def extract_video_id(filename):
    return filename.name.split(" - ")[0]  # everything before the first " - "

# Match files to metadata
filtered_metadata = {}

for file in downloaded_files:
    vid = extract_video_id(file)
    if vid in videos:
        entry = videos[vid]
        timestamp_entry = timestamps_db.get("v_" + vid, {})
        filtered_metadata[vid] = {
            "video_id": vid,
            "duration": entry.get("duration", None),
            "subset": entry.get("subset", ""),
            "annotations": entry.get("annotations", []),
            "timestamps": timestamp_entry.get("timestamps", []),
            "sentences": timestamp_entry.get("sentences", []),
            "file_path": str(file)
        }

# Preview one entry
sample_vid = random.choice(list(filtered_metadata.keys()))
pprint.pprint(filtered_metadata[sample_vid])


{'annotations': [{'label': 'Snatch', 'segment': [0.2, 220.405]},
                 {'label': 'Snatch',
                  'segment': [32.350811232449296, 46.11711388455538]},
                 {'label': 'Snatch', 'segment': [0.2, 220.405]},
                 {'label': 'Snatch', 'segment': [0.2, 220.405]},
                 {'label': 'Snatch', 'segment': [0.2, 220.405]},
                 {'label': 'Snatch', 'segment': [0.2, 220.405]},
                 {'label': 'Snatch', 'segment': [0.2, 220.405]},
                 {'label': 'Snatch', 'segment': [0.2, 220.405]},
                 {'label': 'Snatch', 'segment': [0.2, 220.405]},
                 {'label': 'Snatch', 'segment': [0.2, 220.405]},
                 {'label': 'Snatch', 'segment': [0.2, 220.405]},
                 {'label': 'Snatch', 'segment': [0.2, 220.405]},
                 {'label': 'Snatch', 'segment': [0.2, 220.405]},
                 {'label': 'Snatch', 'segment': [0.2, 220.405]},
                 {'label': 'Snatch',
          

In [64]:
# Convert filtered_metadata to a DataFrame for visualization
df = pd.DataFrame(filtered_metadata.values())

# Display key columns (omit annotations for readability)
preview_df = df[["video_id", "duration", "subset", "annotations", "timestamps", "sentences", "file_path"]]

# Show the table in your notebook
display(preview_df)

,video_id,duration,subset,annotations,timestamps,sentences,file_path
0,-ap649M020k,212.927,training,"[{'segment': [9.965381472401754, 10.9619196196...","[[3.19, 59.62], [56.43, 161.82], [154.37, 199....",[Several clips are shown of people wandering a...,downloads\-ap649M020k - HK Longboarding - Look...
1,74MEdLMrXAQ,202.757,training,"[{'segment': [5.377321265210608, 10.1220164992...","[[4.06, 40.55], [40.55, 95.3], [95.3, 181.47],...",[Boys play indoor soccer while kicking the bal...,downloads\74MEdLMrXAQ - Basic futsal rules.mp4
2,8JKK6JayDeA,220.660,training,"[{'segment': [0.2, 220.405], 'label': 'Snatch'...","[[9.93, 17.65], [24.27, 204.11], [37.51, 204.11]]",[A woman bends down and lifts a heavy weight o...,downloads\8JKK6JayDeA - Olympic Weightlifting ...
3,DLJqhYP-C0k,186.968,training,"[{'segment': [11.083851549980366, 16.625777324...","[[0, 10.28], [10.28, 154.25], [156.12, 163.6],...",[The opening credits are shown for a video abo...,downloads\DLJqhYP-C0k - LOS MEJORES TIROS DE B...
4,jBvGvVw3R-Q,218.620,training,"[{'segment': [19.776733229329174, 23.868471138...","[[0, 27.33], [27.33, 78.7], [78.7, 137.73], [1...",[A man is standing in a gym preparing to lift ...,downloads\jBvGvVw3R-Q - The King of American W...
5,lxMSX0Y0b8I,201.619,training,"[{'segment': [25.751942413517845, 31.090759743...","[[0, 19.15], [19.15, 193.55], [65.53, 76.62], ...","[We see a couple of title screens., We see a ...",downloads\lxMSX0Y0b8I - Movimentos básicos de ...
6,o1WPnnvs00I,229.860,training,"[{'segment': [4.303033313169262, 13.6262721583...","[[0, 16.09], [18.39, 83.9], [86.2, 168.94], [1...",[A man is playing a flute in front of a microp...,downloads\o1WPnnvs00I - Irish March (Fife & Fl...
7,oGwn4NUeoy8,153.090,training,"[{'segment': [37.01843986684637, 42.0338413971...","[[0, 43.63], [32.15, 117.11], [81.9, 146.97]]",[A small group of people are seen on a stage g...,downloads\oGwn4NUeoy8 - Jessica Ryckewaert pla...
8,qF3EbR8y8go,204.100,training,"[{'segment': [2.865726384157407, 9.55242128052...","[[0, 45.92], [45.92, 56.13], [56.13, 93.89], [...",[woman is painting in a white paper green leav...,downloads\qF3EbR8y8go - Chinese Brush Painting...
9,VEDRmPt_-Ms,232.080,training,"[{'segment': [15.568780241809671, 21.723879407...","[[0, 205.39], [205.39, 232.08]]",[women are in a gymnastics competition doing h...,downloads\VEDRmPt_-Ms - Raquel.Pinto.Estrela.M...


Saving filtered metadata to a json file

In [65]:
# Save to JSONL format
with open("filtered_video_metadata.jsonl", "w", encoding="utf-8") as f:
    for item in filtered_metadata.values():
        f.write(json.dumps(item) + "\n")

## Storing video data

In [66]:
import pprint as pp
import requests

host = 'api.novasearch.org'
port = 443

user = 'user12' # Add your user name here.
password = 'black2025yellow' # Add your user password here. For testing only. Don't store credentials in code. 
index_name = user

In [67]:
import pprint as pp
from opensearchpy import OpenSearch
from opensearchpy import helpers

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = (user, password),
    use_ssl = True,
    url_prefix = 'opensearch_v2',
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

if client.indices.exists(index_name):

    resp = client.indices.open(index = index_name)
    print(resp)

    print('\n----------------------------------------------------------------------------------- INDEX SETTINGS')
    settings = client.indices.get_settings(index = index_name)
    pp.pprint(settings)

    print('\n----------------------------------------------------------------------------------- INDEX MAPPINGS')
    mappings = client.indices.get_mapping(index = index_name)
    pp.pprint(mappings)

    print('\n----------------------------------------------------------------------------------- INDEX #DOCs')
    print(client.count(index = index_name))
else:
    print("Index does not exist.")


Index does not exist.


In [68]:
import json
from opensearchpy import OpenSearch, helpers

with open("filtered_video_metadata.jsonl", "r", encoding="utf-8") as f:
    lines = f.readlines()

# Build documents for indexing
docs = []
for line in lines:
    item = json.loads(line)
    
    annotations = " ".join(set([ann["label"] for ann in item.get("annotations", [])]))
    sentences = " ".join(item.get("sentences", []))
    contents = f"{sentences} {annotations}".strip()

    # Add optional tags based on label(s)
    tags = list(set([ann["label"] for ann in item.get("annotations", [])]))

    doc = {
        "_index": index_name,
        "_id": item["video_id"],
        "_source": {
            "doc_id": item["video_id"],
            "contents": contents,
            "tags": tags,
            "json": item
        }
    }
    docs.append(doc)

# Push to OpenSearch
helpers.bulk(client, docs)
print(f"Indexed {len(docs)} videos into {index_name}")

Indexed 10 videos into user12


In [71]:
print(docs)

[{'_index': 'user12', '_id': '-ap649M020k', '_source': {'doc_id': '-ap649M020k', 'contents': 'Several clips are shown of people wandering around areas, looking to the camera, and skateboarding down a hill.  Several more people are shown skateboarding own various areas as well as swimming, rolling joints, and and surfing on the water.  The men perform tricks, get naked, swim underwater, cut open a watermelon, as well as various other activities that lead back into skateboarding. Longboarding', 'tags': ['Longboarding'], 'json': {'video_id': '-ap649M020k', 'duration': 212.927, 'subset': 'training', 'annotations': [{'segment': [9.965381472401754, 10.961919619641929], 'label': 'Longboarding'}, {'segment': [15.280251591016023, 32.88575885892579], 'label': 'Longboarding'}, {'segment': [37.86844959512666, 44.84421662580789], 'label': 'Longboarding'}, {'segment': [54.80959809820965, 60.456647599237314], 'label': 'Longboarding'}, {'segment': [72.7472847485328, 74.07600227818637], 'label': 'Longb

In [74]:
query = "Several"

response = client.search(
    index=index_name,
    body={
        "size": 5,
        "query": {
            "match": {
                "contents": query
            }
        }
    }
)

for hit in response["hits"]["hits"]:
    print(f"Video: {hit['_source']['doc_id']} | Score: {hit['_score']}")
    print(f"Tags: {hit['_source']['tags']}")

Video: -ap649M020k | Score: 1.2184014
Tags: ['Longboarding']
Video: 8JKK6JayDeA | Score: 1.1449025
Tags: ['Snatch']
Video: DLJqhYP-C0k | Score: 0.8826504
Tags: ['Playing ten pins']
Video: jBvGvVw3R-Q | Score: 0.77505386
Tags: ['Snatch']


In [70]:
# if client.indices.exists(index=index_name):
#     # Delete the index.
#     response = client.indices.delete(
#         index = index_name
#     )
#     print('\nDeleting index:')
#     print(response)